In [2]:
pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993196 sha256=97543b7b6dee587fb0c26f8ba58eff629f6d149260b6f5deb7347b555a44aba0
  Stored in directory: c:\users\francesca.casarano\appdata\local\pip\cache\wheels\1e\80\23\0a24928ec3a3906ff5027f38d2fea824e7e97f2ba7c83d91e3
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
from urllib.request import urlopen
import os
import regex as re
from langdetect import detect
import csv
import pandas as pd

In [93]:
#function to parse the html pages
def parse_pages(parent_dir):
    '''
    function to parse the downloded pages and take the infos we need
    input: parent_dir with the downloaded pages
    output: .tsv files
    '''
    article = [] #to create .tsv file
    parent_dir = "E:/HW3"
    
    p = 4        #pages, starting from 1
    n = 319
    while (p<=150): #we start opening the folder related to a specific page
        directory = "HTMLSources_Page"+str(p) #directory of the page
        path = os.path.join(parent_dir, directory) #create path
        os.chdir(path)
        
        book_id = 19
        
        #book id per page, starting from 1
        while(book_id <= 100):
            book = "Book"+str(book_id)+"_"+"Page"+str(p)
            file = open(book, 'r')
            html_doc =  BeautifulSoup(file.read(), features="lxml")
            
            if html_doc.find_all('h1'):
                #print(html_doc.find(id = 'bookTitle'))
                bookTitle = html_doc.find_all('h1')[0].contents[0]
                bookTitle = bookTitle.split('\n')[1].strip()
                #bookTitle = html_doc.find(id = 'bookTitle').contents
                #print(bookTitle)
                article.append(bookTitle)
            else:
                print('Check this book','Book:', book_id ,'at page:', p )
                
            if html_doc.find_all('span', itemprop="name"): 
                bookAuthors = html_doc.find_all('span', itemprop="name")[0].contents[0]
                article.append(bookAuthors)
            
            #h2 id="bookSeries"
            #bookSeries = html_doc.find('h2', id="bookSeries").contents[1].contents[0]
            #print(bookSeries)
            if html_doc.find_all('a', class_="greyText"):
                bookSeries = html_doc.find_all('a', class_="greyText")[0].contents[0]
                if bookSeries == 'Edit Details':
                    bookSeries = ''
                    article.append(bookSeries)
                else:
                    bookSeries = bookSeries.split('\n')[1].strip()
                    article.append(bookSeries)
            #print(bookSeries)
            
            #ratingValue = html_doc.find_all('span', itemprop="ratingValue")[0].contents[0]
            #ratingValue = ratingValue.split('\n')[1].strip()
            if html_doc.find(itemprop="ratingValue"):
                ratingValue = html_doc.find(itemprop="ratingValue").text
                ratingValue = ratingValue.split('\n')[1].strip()
                article.append(ratingValue)
            if html_doc.find(itemprop="ratingCount"):
                #print(html_doc.find(itemprop="ratingCount").get('content'))
                ratingCount = html_doc.find(itemprop="ratingCount").get('content')
                #ratingCount = ratingCount.split('\n')[1].strip()
                article.append(ratingCount)
            if html_doc.find(itemprop="reviewCount"):
                reviewCount = html_doc.find(itemprop="reviewCount").get('content')
                #reviewCount = reviewCount.split('\n')[1].strip()
                article.append(reviewCount)
                
            
            #print(' '.join([c for c in html_doc.find_all('div', id="description")[0].contents[3] if isinstance(c, str)]))
            #print(html_doc.find(id="description"))
            #print(html_doc.find_all('div', id="description")[0].contents[3].text)
            
            #print(html_doc.find_all('div', id="description"))
                #print( (html_doc.find(id="description").find_all('span')[-1]).text)
                #print(' '.join([c.text for c in html_doc.find_all('div', id="description")[0].find_all('span')[-1]]))
                #print(html_doc.find_all('div', id="description")[0].find_all('span')[-1].text)
                #print(item.text for item in (html_doc.find( id="description").find_all('span')[1]))
                #Plot = ' '.join([c.text for c in html_doc.find_all(id="description")[0].find_all('span')[-1] ])
                #print(Plot)
                #Plot = html_doc.find_all('div', id="description")[0].contents[3].text
            if html_doc.find_all('div', id="description"):
                Plot = (html_doc.find(id="description").find_all('span')[-1]).text
                article.append(Plot)
            else:
                Plot = ''
                article.append(Plot)
            
            if (html_doc.find_all('span', {'itemprop' : "numberOfPages"})):      
                NumberofPages = html_doc.find_all('span', itemprop="numberOfPages")[0].contents[0]
                NumberofPages = NumberofPages.split(' ')[0]
                article.append(NumberofPages)
            else:
                NumberofPages = ''
                article.append(NumberofPages)
            
            if (html_doc.find_all('div', {'class' : "row"})):
                PublishingDate = html_doc.find_all('div', class_="row")[1].contents[0]
                if (len(PublishingDate.split('\n')) >= 3):
                    PublishingDate = PublishingDate.split('\n')[2].strip()
                    article.append(PublishingDate)
                else:
                    PublishingDate = ""
                    article.append(PublishingDate)
            else:
                PublishingDate = ""    
                article.append(PublishingDate)
            
            if html_doc.find_all('a', {'href' : re.compile(r'/characters')}):
                Characters = ','.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/characters'))])
                article.append(Characters)
            else:
                Characters = ""
                article.append(Characters)
            
            if html_doc.find_all('a', {'href' : re.compile(r'/places')}):
                Setting = ' '.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/places'))]) 
                article.append(Setting)
            else:
                Setting = ""
                article.append(Setting)
            
            if html_doc.find('link'):
                Url = html_doc.find('link').get('href')
                article.append(Url)
                #print(Url)
            #print(article)
            
            
            try: 
                detect(Plot) =='eng'
                #article.append(Plot)
            
                path = 'E:\\HW3\\Articles'
                #name_article = "articles.tsv"
                name_article = "article_"+str(n)+".tsv"
                    #path = 'E:\\Master-Data Science\\ADM\\HW3\\Articles'
                    #name_article = "article_"+str(n)+".tsv"
        
                with open(os.path.join(path,name_article), 'w', newline='') as f_output:
                    tsv_output = csv.writer(f_output, delimiter='\t')
                    tsv_output.writerow(article)
                    n += 1
                    article.clear()
                    #article.append()
            except:
                print('no plot','Page:', p ,'Book:', book_id )
                
            book_id +=1
        p +=1
            
#####################################################        
article = [] #to create .tsv file
parent_dir = "E:/HW3"
parse_pages(parent_dir)

Check this book Book: 19 at page: 4
no plot Page: 4 Book: 19


KeyboardInterrupt: 

In [5]:
try:
    detect(' ')
except:
    print(' spaces no text')

 spaces no text


In [101]:
#function to parse the html pages
def parse_pages(parent_dir):
    '''
    function to parse the downloded pages and take the infos we need
    input: parent_dir with the downloaded pages
    output: .tsv files
    '''
    article = [] #to create .tsv file
    parent_dir = "E:/HW3"
    
    p = 1        #pages, starting from 1
    n = 1
    
    while (p<=150): #we start opening the folder related to a specific page
        directory = "HTMLSources_Page"+str(p) #directory of the page
        path = os.path.join(parent_dir, directory) #create path
        os.chdir(path)
        
        book_id = 1
        
        #book id per page, starting from 1
        while(book_id <= 100):
            book = "Book"+str(book_id)+"_"+"Page"+str(p)
            file = open(book, 'r')
            html_doc =  BeautifulSoup(file.read(), features="lxml")
                
            if html_doc.find_all('h1'):
                if len(html_doc.find_all('h1')[0]) > 0:
                    #print(html_doc.find(id = 'bookTitle'))
                    bookTitle = html_doc.find_all('h1')[0].contents[0]
                    bookTitle = bookTitle.split('\n')[1].strip()
                    #bookTitle = html_doc.find(id = 'bookTitle').contents
                    #print(bookTitle)
                    article.append(bookTitle)
                
                    if html_doc.find_all('span', itemprop="name"): 
                        if len(html_doc.find_all('span', itemprop="name")[0]) > 0: #prova a vedere se è sufficiente tenere questo!
                            bookAuthors = html_doc.find_all('span', itemprop="name")[0].contents[0]
                            article.append(bookAuthors)
                            #print(html_doc.find_all('span', itemprop="name")[0].contents)
                            #bookAuthors = html_doc.find_all('span', itemprop="name")[0].contents[0]
                            #if len(bookAuthors) != 0:
                            #article.append(bookAuthors)
        
                            if html_doc.find_all('a', class_="greyText"):
                                bookSeries = html_doc.find_all('a', class_="greyText")[0].contents[0]
                                if bookSeries == 'Edit Details':
                                    bookSeries = ''
                                    article.append(bookSeries)
                                else:
                                    bookSeries = bookSeries.split('\n')[1].strip()
                                    article.append(bookSeries)
            
                            if html_doc.find(itemprop="ratingValue"):
                                ratingValue = html_doc.find(itemprop="ratingValue").text
                                ratingValue = ratingValue.split('\n')[1].strip()
                                article.append(ratingValue)
                            if html_doc.find(itemprop="ratingCount"):
                                #print(html_doc.find(itemprop="ratingCount").get('content'))
                                ratingCount = html_doc.find(itemprop="ratingCount").get('content')
                                #ratingCount = ratingCount.split('\n')[1].strip()
                                article.append(ratingCount)
                            if html_doc.find(itemprop="reviewCount"):
                                reviewCount = html_doc.find(itemprop="reviewCount").get('content')
                                #reviewCount = reviewCount.split('\n')[1].strip()
                                article.append(reviewCount)
            
                            if html_doc.find_all('div', id="description"):
                                Plot = (html_doc.find(id="description").find_all('span')[-1]).text
                                article.append(Plot)
                            else:
                                Plot = ''
                                article.append(Plot)
            
                            if (html_doc.find_all('span', {'itemprop' : "numberOfPages"})):      
                                NumberofPages = html_doc.find_all('span', itemprop="numberOfPages")[0].contents[0]
                                NumberofPages = NumberofPages.split(' ')[0]
                                article.append(NumberofPages)
                            else:
                                NumberofPages = ''
                                article.append(NumberofPages)
            
                            if (html_doc.find_all('div', {'class' : "row"})):
                                PublishingDate = html_doc.find(id="details").find_all('div')[1].text
                                PublishingDate = PublishingDate.split('\n')[2].strip()
                                article.append(PublishingDate)
                            else:
                                PublishingDate = ""    
                                article.append(PublishingDate)
            
                            if html_doc.find_all('a', {'href' : re.compile(r'/characters')}):
                                Characters = ','.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/characters'))])
                                article.append(Characters)
                            else:
                                Characters = ""
                                article.append(Characters)
            
                            if html_doc.find_all('a', {'href' : re.compile(r'/places')}):
                                Setting = ' '.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/places'))]) 
                                article.append(Setting)
                            else:
                                Setting = ""
                                article.append(Setting)
            
                            if html_doc.find('link'):
                                Url = html_doc.find('link').get('href')
                                article.append(Url)
                                
            
                            try: 
                                detect(Plot) =='eng'
                                #article.append(Plot)
            
                                path = 'E:\\HW3\\Articles_3'
                                #path = 'E:\\Master-Data Science\\ADM\\HW3\\Articles'
                                name_article = "article_"+str(n)+".tsv"
            
                                with open(os.path.join(path,name_article), 'w', newline='') as f_output:
                                    tsv_output = csv.writer(f_output, delimiter='\t')
                                    tsv_output.writerow(article)
                                    n += 1
                                    article.clear()
                                    #article.append()
                            except:
                                print('Plot non accettato del libro', book_id, 'a pagina :', p )
            
            print('Siamo al libro', book_id, 'di pagina', p)
            
            book_id +=1
            
        p +=1
            
#####################################################        
article = [] #to create .tsv file
parent_dir = "E:/HW3"
parse_pages(parent_dir)

Siamo al libro 1 di pagina 1
Siamo al libro 2 di pagina 1
Siamo al libro 3 di pagina 1
Siamo al libro 4 di pagina 1
Siamo al libro 5 di pagina 1
Siamo al libro 6 di pagina 1
Siamo al libro 7 di pagina 1
Siamo al libro 8 di pagina 1
Siamo al libro 9 di pagina 1
Siamo al libro 10 di pagina 1
Siamo al libro 11 di pagina 1
Siamo al libro 12 di pagina 1
Siamo al libro 13 di pagina 1
Siamo al libro 14 di pagina 1
Siamo al libro 15 di pagina 1
Siamo al libro 16 di pagina 1
Siamo al libro 17 di pagina 1
Siamo al libro 18 di pagina 1
Siamo al libro 19 di pagina 1
Siamo al libro 20 di pagina 1
Siamo al libro 21 di pagina 1
Siamo al libro 22 di pagina 1
Siamo al libro 23 di pagina 1
Siamo al libro 24 di pagina 1
Siamo al libro 25 di pagina 1
Siamo al libro 26 di pagina 1
Siamo al libro 27 di pagina 1
Siamo al libro 28 di pagina 1
Siamo al libro 29 di pagina 1
Siamo al libro 30 di pagina 1
Siamo al libro 31 di pagina 1
Siamo al libro 32 di pagina 1
Siamo al libro 33 di pagina 1
Siamo al libro 34 d

Siamo al libro 75 di pagina 3
Siamo al libro 76 di pagina 3
Siamo al libro 77 di pagina 3
Siamo al libro 78 di pagina 3
Siamo al libro 79 di pagina 3
Siamo al libro 80 di pagina 3
Siamo al libro 81 di pagina 3
Siamo al libro 82 di pagina 3
Siamo al libro 83 di pagina 3
Siamo al libro 84 di pagina 3
Siamo al libro 85 di pagina 3
Siamo al libro 86 di pagina 3
Siamo al libro 87 di pagina 3
Siamo al libro 88 di pagina 3
Siamo al libro 89 di pagina 3
Siamo al libro 90 di pagina 3
Siamo al libro 91 di pagina 3
Siamo al libro 92 di pagina 3
Siamo al libro 93 di pagina 3
Siamo al libro 94 di pagina 3
Plot non accettato del libro 95 a pagina : 3
Siamo al libro 95 di pagina 3
Siamo al libro 96 di pagina 3
Siamo al libro 97 di pagina 3
Siamo al libro 98 di pagina 3
Siamo al libro 99 di pagina 3
Siamo al libro 100 di pagina 3
Siamo al libro 1 di pagina 4
Siamo al libro 2 di pagina 4
Siamo al libro 3 di pagina 4
Siamo al libro 4 di pagina 4
Siamo al libro 5 di pagina 4
Siamo al libro 6 di pagina 4


Siamo al libro 45 di pagina 6
Siamo al libro 46 di pagina 6
Siamo al libro 47 di pagina 6
Siamo al libro 48 di pagina 6
Siamo al libro 49 di pagina 6
Siamo al libro 50 di pagina 6
Siamo al libro 51 di pagina 6
Siamo al libro 52 di pagina 6
Siamo al libro 53 di pagina 6
Siamo al libro 54 di pagina 6
Siamo al libro 55 di pagina 6
Siamo al libro 56 di pagina 6
Siamo al libro 57 di pagina 6
Siamo al libro 58 di pagina 6
Siamo al libro 59 di pagina 6
Siamo al libro 60 di pagina 6
Siamo al libro 61 di pagina 6
Siamo al libro 62 di pagina 6
Siamo al libro 63 di pagina 6
Siamo al libro 64 di pagina 6
Siamo al libro 65 di pagina 6
Siamo al libro 66 di pagina 6
Siamo al libro 67 di pagina 6
Siamo al libro 68 di pagina 6
Siamo al libro 69 di pagina 6
Siamo al libro 70 di pagina 6
Siamo al libro 71 di pagina 6
Siamo al libro 72 di pagina 6
Siamo al libro 73 di pagina 6
Siamo al libro 74 di pagina 6
Siamo al libro 75 di pagina 6
Siamo al libro 76 di pagina 6
Siamo al libro 77 di pagina 6
Siamo al l

Siamo al libro 15 di pagina 9
Siamo al libro 16 di pagina 9
Siamo al libro 17 di pagina 9
Siamo al libro 18 di pagina 9
Siamo al libro 19 di pagina 9
Siamo al libro 20 di pagina 9
Siamo al libro 21 di pagina 9
Siamo al libro 22 di pagina 9
Siamo al libro 23 di pagina 9
Siamo al libro 24 di pagina 9
Siamo al libro 25 di pagina 9
Siamo al libro 26 di pagina 9
Siamo al libro 27 di pagina 9
Siamo al libro 28 di pagina 9
Siamo al libro 29 di pagina 9
Siamo al libro 30 di pagina 9
Siamo al libro 31 di pagina 9
Siamo al libro 32 di pagina 9
Siamo al libro 33 di pagina 9
Siamo al libro 34 di pagina 9
Siamo al libro 35 di pagina 9
Siamo al libro 36 di pagina 9
Siamo al libro 37 di pagina 9
Siamo al libro 38 di pagina 9
Siamo al libro 39 di pagina 9
Siamo al libro 40 di pagina 9
Siamo al libro 41 di pagina 9
Siamo al libro 42 di pagina 9
Siamo al libro 43 di pagina 9
Siamo al libro 44 di pagina 9
Siamo al libro 45 di pagina 9
Siamo al libro 46 di pagina 9
Siamo al libro 47 di pagina 9
Siamo al l

Siamo al libro 77 di pagina 11
Siamo al libro 78 di pagina 11
Siamo al libro 79 di pagina 11
Siamo al libro 80 di pagina 11
Siamo al libro 81 di pagina 11
Siamo al libro 82 di pagina 11
Siamo al libro 83 di pagina 11
Siamo al libro 84 di pagina 11
Siamo al libro 85 di pagina 11
Siamo al libro 86 di pagina 11
Siamo al libro 87 di pagina 11
Siamo al libro 88 di pagina 11
Siamo al libro 89 di pagina 11
Siamo al libro 90 di pagina 11
Siamo al libro 91 di pagina 11
Siamo al libro 92 di pagina 11
Siamo al libro 93 di pagina 11
Siamo al libro 94 di pagina 11
Siamo al libro 95 di pagina 11
Siamo al libro 96 di pagina 11
Siamo al libro 97 di pagina 11
Siamo al libro 98 di pagina 11
Siamo al libro 99 di pagina 11
Siamo al libro 100 di pagina 11
Siamo al libro 1 di pagina 12
Siamo al libro 2 di pagina 12
Siamo al libro 3 di pagina 12
Siamo al libro 4 di pagina 12
Siamo al libro 5 di pagina 12
Siamo al libro 6 di pagina 12
Siamo al libro 7 di pagina 12
Siamo al libro 8 di pagina 12
Siamo al libro 

...

Siamo al libro 27 di pagina 140
Siamo al libro 28 di pagina 140
Siamo al libro 29 di pagina 140
Siamo al libro 30 di pagina 140
Siamo al libro 31 di pagina 140
Siamo al libro 32 di pagina 140
Siamo al libro 33 di pagina 140
Siamo al libro 34 di pagina 140
Siamo al libro 35 di pagina 140
Siamo al libro 36 di pagina 140
Siamo al libro 37 di pagina 140
Siamo al libro 38 di pagina 140
Siamo al libro 39 di pagina 140
Siamo al libro 40 di pagina 140
Siamo al libro 41 di pagina 140
Siamo al libro 42 di pagina 140
Siamo al libro 43 di pagina 140
Siamo al libro 44 di pagina 140
Siamo al libro 45 di pagina 140
Siamo al libro 46 di pagina 140
Siamo al libro 47 di pagina 140
Siamo al libro 48 di pagina 140
Siamo al libro 49 di pagina 140
Siamo al libro 50 di pagina 140
Siamo al libro 51 di pagina 140
Siamo al libro 52 di pagina 140
Siamo al libro 53 di pagina 140
Siamo al libro 54 di pagina 140
Siamo al libro 55 di pagina 140
Siamo al libro 56 di pagina 140
Siamo al libro 57 di pagina 140
Siamo al

Siamo al libro 68 di pagina 142
Siamo al libro 69 di pagina 142
Plot non accettato del libro 70 a pagina : 142
Siamo al libro 70 di pagina 142
Siamo al libro 71 di pagina 142
Siamo al libro 72 di pagina 142
Siamo al libro 73 di pagina 142
Siamo al libro 74 di pagina 142
Siamo al libro 75 di pagina 142
Siamo al libro 76 di pagina 142
Siamo al libro 77 di pagina 142
Siamo al libro 78 di pagina 142
Siamo al libro 79 di pagina 142
Siamo al libro 80 di pagina 142
Siamo al libro 81 di pagina 142
Siamo al libro 82 di pagina 142
Siamo al libro 83 di pagina 142
Siamo al libro 84 di pagina 142
Siamo al libro 85 di pagina 142
Siamo al libro 86 di pagina 142
Siamo al libro 87 di pagina 142
Siamo al libro 88 di pagina 142
Siamo al libro 89 di pagina 142
Siamo al libro 90 di pagina 142
Siamo al libro 91 di pagina 142
Siamo al libro 92 di pagina 142
Siamo al libro 93 di pagina 142
Siamo al libro 94 di pagina 142
Siamo al libro 95 di pagina 142
Siamo al libro 96 di pagina 142
Siamo al libro 97 di pagi

Siamo al libro 15 di pagina 145
Siamo al libro 16 di pagina 145
Siamo al libro 17 di pagina 145
Siamo al libro 18 di pagina 145
Siamo al libro 19 di pagina 145
Siamo al libro 20 di pagina 145
Siamo al libro 21 di pagina 145
Siamo al libro 22 di pagina 145
Siamo al libro 23 di pagina 145
Siamo al libro 24 di pagina 145
Siamo al libro 25 di pagina 145
Siamo al libro 26 di pagina 145
Siamo al libro 27 di pagina 145
Siamo al libro 28 di pagina 145
Siamo al libro 29 di pagina 145
Siamo al libro 30 di pagina 145
Siamo al libro 31 di pagina 145
Siamo al libro 32 di pagina 145
Siamo al libro 33 di pagina 145
Siamo al libro 34 di pagina 145
Siamo al libro 35 di pagina 145
Siamo al libro 36 di pagina 145
Siamo al libro 37 di pagina 145
Siamo al libro 38 di pagina 145
Siamo al libro 39 di pagina 145
Siamo al libro 40 di pagina 145
Siamo al libro 41 di pagina 145
Siamo al libro 42 di pagina 145
Siamo al libro 43 di pagina 145
Siamo al libro 44 di pagina 145
Siamo al libro 45 di pagina 145
Plot non

Siamo al libro 57 di pagina 147
Siamo al libro 58 di pagina 147
Siamo al libro 59 di pagina 147
Siamo al libro 60 di pagina 147
Siamo al libro 61 di pagina 147
Siamo al libro 62 di pagina 147
Siamo al libro 63 di pagina 147
Siamo al libro 64 di pagina 147
Siamo al libro 65 di pagina 147
Siamo al libro 66 di pagina 147
Siamo al libro 67 di pagina 147
Siamo al libro 68 di pagina 147
Siamo al libro 69 di pagina 147
Siamo al libro 70 di pagina 147
Siamo al libro 71 di pagina 147
Siamo al libro 72 di pagina 147
Siamo al libro 73 di pagina 147
Siamo al libro 74 di pagina 147
Plot non accettato del libro 75 a pagina : 147
Siamo al libro 75 di pagina 147
Siamo al libro 76 di pagina 147
Siamo al libro 77 di pagina 147
Siamo al libro 78 di pagina 147
Siamo al libro 79 di pagina 147
Siamo al libro 80 di pagina 147
Siamo al libro 81 di pagina 147
Siamo al libro 82 di pagina 147
Siamo al libro 83 di pagina 147
Siamo al libro 84 di pagina 147
Siamo al libro 85 di pagina 147
Siamo al libro 86 di pagi

Siamo al libro 8 di pagina 150
Siamo al libro 9 di pagina 150
Siamo al libro 10 di pagina 150
Siamo al libro 11 di pagina 150
Siamo al libro 12 di pagina 150
Siamo al libro 13 di pagina 150
Siamo al libro 14 di pagina 150
Plot non accettato del libro 15 a pagina : 150
Siamo al libro 15 di pagina 150
Siamo al libro 16 di pagina 150
Siamo al libro 17 di pagina 150
Siamo al libro 18 di pagina 150
Siamo al libro 19 di pagina 150
Siamo al libro 20 di pagina 150
Siamo al libro 21 di pagina 150
Siamo al libro 22 di pagina 150
Siamo al libro 23 di pagina 150
Siamo al libro 24 di pagina 150
Siamo al libro 25 di pagina 150
Siamo al libro 26 di pagina 150
Siamo al libro 27 di pagina 150
Siamo al libro 28 di pagina 150
Siamo al libro 29 di pagina 150
Siamo al libro 30 di pagina 150
Siamo al libro 31 di pagina 150
Siamo al libro 32 di pagina 150
Siamo al libro 33 di pagina 150
Siamo al libro 34 di pagina 150
Siamo al libro 35 di pagina 150
Siamo al libro 36 di pagina 150
Siamo al libro 37 di pagina